# 📌 XGBoost

In [ ]:
# python black formatting
%reload_ext nb_black
%reload_ext lab_black

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

import os
import random
import eli5
import warnings

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, accuracy_score
from eli5.sklearn import PermutationImportance
from xgboost import XGBClassifier, plot_importance

warnings.filterwarnings(action="ignore")  # 경고 출력 무시

## ✅ 데이터 로딩

In [ ]:
DATA_PATH = "/opt/ml/input/data/all_feature_data.csv"
GT_DATA_PATH = "/opt/ml/input/data/ground_truth.csv"

# train data 불러오기
df = pd.read_csv(DATA_PATH, parse_dates=["Timestamp"])
df = df.sort_values(by=["userID", "Timestamp"]).reset_index(drop=True)

# ground truth 불러오기
gt = pd.read_csv(GT_DATA_PATH)["gt"]

## ✅ Feature Engineering
- Special mission의 Feature Engineering 코드
- Category feature의 변환

In [ ]:
# 카테고리형 변수 설정하는 함수
def feature_engineering(df):

    # 카테고리형 feature
    categories = [
        "assessmentItemID",
        "testId",
    ]

    # label encode your categorical columns
    le = preprocessing.LabelEncoder()
    for category in categories:
        df[category] = le.fit_transform(df[category])

    return df


df = feature_engineering(df)

In [ ]:
# train과 test 데이터 나누기
train_df = df[df.dataset == 1]
test_df = df[df.dataset == 2]

## ✅ Train/Test 데이터 셋 분리 (option1, option2에서 하나만 실행)

### 📍 Option 1
- train 데이터에서 train, valid set을 나눔

In [ ]:
# train과 valid 데이터셋은 사용자 별로 묶어서 분리를 해주어야함
random.seed(42)

# 데이터에 포함된 answerCode 비율을 동일하게 맞추어준다.
def make_proportion_equal(df: pd.DataFrame):

    # index를 다시 새롭게 0부터 매긴다. (학습 지장 X)
    df = df.reset_index(drop=True)

    # answerCode가 1 이면 hit, 0이면 error 라고 표현
    error_index = list(df[df.answerCode == 0].index)
    hit_index = list(df[df.answerCode == 1].index)

    # 0의 개수와 1의 개수를 계산
    error_len = len(error_index)
    hit_len = len(hit_index)

    # 더 많은 것은 낮은 것을 기준으로 맞추어준다.
    threshold = min(error_len, hit_len)

    if error_len > threshold:
        random.shuffle(error_index)
        error_index = error_index[:threshold]

    elif hit_len > threshold:
        random.shuffle(hit_index)
        hit_index = hit_index[:threshold]

    # 최종적으로 사용할 데이터 index 들
    error_index.extend(hit_index)

    return df.iloc[error_index]


def option1_train_test_split(df, ratio=0.9, split=True):

    # users = [userID, user의 interaction 수]
    users = list(zip(df["userID"].value_counts().index, df["userID"].value_counts()))
    random.shuffle(users)

    # 전체 interaction의 ratio만큼 train_dataset으로 사용
    # 유저가 train, valid로 분리되어서는 안되기 때문에 묶어서 연산을 진행
    max_train_data_len = ratio * len(df)
    sum_of_train_data = 0
    train_users = []

    # train_data 에 포함될 user 추출
    for user_id, count in users:
        sum_of_train_data += count
        if max_train_data_len < sum_of_train_data:
            break
        train_users.append(user_id)

    # train dataset, valid dataset 생성
    train = df[df["userID"].isin(train_users)]
    valid = df[df["userID"].isin(train_users) == False]

    # test데이터셋은 각 유저의 마지막 interaction만 추출
    valid = valid[valid["userID"] != valid["userID"].shift(-1)]

    # 🎯 만약 비율을 같게 만들고 싶다면 사용 (데이터 감소되는 단점)
    # train = make_proportion_equal(train)

    return train, valid


# 유저 별 분리
train, valid = option1_train_test_split(train_df)

### 📍 Option 2
- train 데이터를 모두 훈련에 사용
- valid를 test셋의 마지막 두번째 데이터로 진행

In [ ]:
def option2_train_test_split(df):
    # use train dataset only for train
    train = df[df.dataset == 1]

    # use test dataset only for valid
    test = df[(df.dataset == 2) & (df.answerCode != -1)]  # -1 인 answerCode 제외

    # test데이터셋은 각 유저의 마지막 interaction만 추출
    test = test[test["userID"] != test["userID"].shift(-1)]

    return train, test


train, valid = option2_train_test_split(df)

## ✅ 데이터셋 정의

In [ ]:
# 학습에 사용하는 feature와 최종으로 맞추어야 하는 값 설정
y_train = train["answerCode"]
train = train.drop(["answerCode"], axis=1)

y_valid = valid["answerCode"]
valid = valid.drop(["answerCode"], axis=1)

In [ ]:
# dataset에 포함된 feature 목록
sorted(list(train.columns))

- FEATS 에 사용할 feature를 설정

In [ ]:
FEATS = [
    # "KTAccuracyCate",
    "KnowledgeTag",
    # "KnowledgeTagAcc",
    # "Timestamp",
    "accuracy",
    "assessmentItemID",
    "bigClass",
    "bigClassAcc",
    "bigClassAccCate",
    "bigClassElaspedTimeAvg",
    "cumAccuracy",
    "cumCorrect",
    "dataset",
    "day",
    # "elapsedTime",
    "elapsedTimeClass",
    "elapsedTime_ver2",
    "elo",
    "hour",
    "month",
    "problemNumber",
    "recAccuracy",
    # "recCount",
    "seenCount",
    "tagCluster",
    "tagCount",
    # "tagLV",
    # "testId",
    # "testLV",
    "userID",
    # "userLVbyTag",
    "userLVbyTagAVG",
    # "userLVbyTest",
    "userLVbyTestAVG",
    "wday",
    "weekNum",
    "year",
    "z-time",
]

## ✅ 훈련 및 검증

### 🔍 XGBoost Parameter 설명

[1] `booster` : 'gbtree'가 항상 더 좋은 성능을 보이기 때문에 수정할 필요 X  
[2] `colsample_bylevel` : 트리의 레벨별로 훈련 데이터의 변수를 샘플링해주는 비율. 보통 (0.6 ~ 0.9)  
[3] `colsample_bytree` : 트리를 생성할 때 훈련 데이터에서 변수를 샘플링해주는 비율. 보통 (0.6 ~ 0.9)  
[4] `gamma` : 노드가 split 되기 위한 loss function 값이 감소하는 최소값. gamma값이 커지면 알고리즘은 보수적으로 변하고, loss function의 정의에 따라 적정값이 달라지기 때문에 반드시 튜닝 필요!!   
[5] `max_depth` : 트리의 최대 깊이 지정. 일반적으로 3 ~ 10 사용  
[6] `min_child_weight` : 값이 높아지면 under-fitting 될 수 있다. CV를 통한 튜닝 필요  
[7] `n_estimators` : Boost 트리의 수  
[8] `nthread` : 병렬처리 스레드의 수  
[9] `objective` : 'binary:logistic'사용   
[10] `random_state` : random seed와 동일  
[11] `silent` :  running method 출력할지 결정   

In [ ]:
# 🌟 XGBoost 모델의 파라미터 설정
model = XGBClassifier(
    booster="gbtree",
    colsample_bylevel=0.8,
    colsample_bytree=0.8,
    gamma=0,
    max_depth=8,
    min_child_weight=4,
    n_estimators=70,
    nthread=4,
    objective="binary:logistic",
    random_state=42,
    # silent=True,
)

# 🌟 XGBoost 모델 학습
model.fit(
    X=train[FEATS],
    y=y_train,
    eval_set=[(valid[FEATS], y_valid)],
    early_stopping_rounds=100,
    verbose=5,
)

In [ ]:
# 🌟 XGBoost 모델 추론
preds = model.predict_proba(valid[FEATS])[:, 1]
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f"VALID AUC : {auc} ACC : {acc}\n")

# feature 중요도 출력
fscore = model.get_booster().get_fscore()
print(sorted(fscore, key=lambda x: x[1]))

plot_importance(model)

## ✅ Inference

In [ ]:
test_df = df[df.dataset == 2]

# LEAVE LAST INTERACTION ONLY
test_df = test_df[test_df["userID"] != test_df["userID"].shift(-1)]

# DROP ANSWERCODE
test_df = test_df.drop(["answerCode"], axis=1)

# MAKE PREDICTION
total_preds = model.predict_proba(test_df[FEATS])[:, 1]

In [ ]:
# SAVE OUTPUT
output_dir = "output/"
write_path = os.path.join(output_dir, "submission.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
with open(write_path, "w", encoding="utf8") as w:
    print("writing prediction : {}".format(write_path))
    w.write("id,prediction\n")
    for id, p in enumerate(total_preds):
        w.write("{},{}\n".format(id, p))

## ✅ Real Submission Score

In [ ]:
pred = pd.read_csv("./output/submission.csv")["prediction"]
answer = gt

acc = accuracy_score(answer, np.where(pred >= 0.5, 1, 0))
auc = roc_auc_score(answer, pred)

print(f"VALID AUC : {auc} ACC : {acc}\n")